In [149]:
import random
import numpy as np
import random
from matplotlib import pyplot as plt

np.random.seed(0)

def greedy(results, n):#方策１
    slot_num = None
    for i,d in enumerate(results):
        if d[1] < n:
            slot_num = i
            break
    if slot_num == None:
        slot_num = np.array([row[3] for row in results]).argmax()
    return slot_num

def epsilon_greedy(results, epsilon):#方策２
    slot_num = None
    for i,d in enumerate(results):
         if d[1] ==0:
            slot_num = i
            break
    if slot_num ==None:#確率条件分岐
        if random.random() < epsilon:
             slot_num = np.random.randint(0,5)
        else:
             slot_num = np.array([row[3] for row in results]).argmax()
    return slot_num

def optimistic(results, k, rsup):#方策3
    optimistic_mean = np.array([(row[2] + k*rsup)/(row[1] + k) for row in results])
    slot_num = optimistic_mean.argmax()
    return slot_num

def softmax(results, tau):#方策4
    q = np.array([row[3] for row in results])
    if q.sum() ==0:
        q += 1
        q = sum(q)
    else:
        q = np.exp(q/tau)/sum(np.exp(q/tau))
    slot_num = np.random.choice([0,1,2,3,4], p=q)      
    return slot_num   

def UCB(results, R):#方策5
    slot_num = None
    for i,d in enumerate(results):
         if d[1] ==0:
            slot_num = i
            break
    if slot_num ==None:
        times = np.array([row[1] for row in results])
        u = np.array([row[3] for row in results])
        x = R*np.sqrt((2*np.log(sum(times)))/times)
        slot_num = (u+x).argmax()
    return slot_num

    
def environments(band_number):#environment only result
    coins_p = np.array([0.3, 0.4, 0.5, 0.6, 0.7])
    results = np.random.binomial(1, coins_p)
    result = results[band_number]
    return result

def reward(record, results, slot_num, time):
    result = environments(slot_num)
    record[time] = result
    results[slot_num][1] += 1
    results[slot_num][2] += result
    results[slot_num][3] = results[slot_num][2] / results[slot_num][1]
    return results, record

times = 10000
results = [[0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0], [4, 0, 0, 0]]
record = np.zeros(times)
n = 100

In [53]:
import numpy as np

state_transition = np.array(
                    [[0, 0, 0, 0.3, 5],
                    [0, 1, 0, 0.7, -10],
                    [0, 0, 1, 1, 5],
                    [1, 0, 1, 1, 5],
                    [1, 2, 0, 0.8, 100],
                    [1, 1, 0, 0.2, -10]]
                    )

In [54]:
def actions(state):#global variable state_trandition
    if terminals ==[2]:
        return None
    A = np.unique(np.array([row[2] for row in state_transition if row[0]==state]))
    return A#取りうるアクション

In [55]:
def R(state, action, after_state):
    if state==terminals:
        return 0
    else:
        reward = np.array([row[4] for row in state_transition 
                           if row[0]==state and row[1]==after_state and row[2]==action])
    return reward[0]#得られた報酬

In [60]:
def T(state, action):
    if state==terminals:
        return [(0, terminals)]
    else:
        return [(row[3], row[1]) for row in state_transition 
                if row[0]==state and row[2]==action]
        #（確率、次の状態）のタプル

[(0.8, 2.0), (0.2, 1.0)]


In [154]:
def take_single_action(state, action):
    x = random.random()
    for p, nextate in T(state, action):
        sump = p
        if x < sump :
            next_state = nextate
            break
    return next_state

In [178]:
def calculate_value():#if all action = actionX
    action = 0
    state = init
    sumg = 0
    time = 0
    while True:
        next_state = take_single_action(state, action)
        sumg += R(state, action, next_state)*(gamma**time)
        time += 1
        state = next_state
        if state==terminals:
            break
    return sumg

In [181]:
state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:, 0])
actlist = np.unique(state_transition[:, 2])
terminals = [2]
gamma = 0.8

V = {s: 0 for s in np.hstack((states,terminals))}
pi = {s: 0 for s in states}

In [242]:
pi

{0.0: 0, 1.0: 0}

In [238]:
take_single_action(1, 1)

0.0

In [240]:
states

array([0., 1.])

In [241]:
V = {0.0: 0, 1.0: 0, 2.0: 0}

In [243]:
def policy_evaluation(pi, states, epsilon=0.001):
    while True:
        delta = 0
        for s in states:
            v = V[s]
            sumv = 0
            for p, after_state in T(s, pi[s]):
                sumv += p*(R(s, pi[s], after_state) + gamma*V[after_state])
            V[s] = sumv
            delta = max(delta,abs(v - V[s]))
        if  delta < epsilon:
            return V

In [244]:
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1, None],
                      [None, -0.04, -0.04, -0.04, +1, None],
                      [None, None, None, None, None, None]])

In [245]:
situation[1,1]

-0.04

In [246]:
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))

In [247]:
actions[0]

(1, 0)

In [250]:
print(situation[actions[0]])

None


In [251]:
q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

In [255]:
q_table[11,1]

0.0034127573923631877